# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
def date_sorter():
    
    df_dates = df.str.extract(r'(\d{1,2}[\/|\-]\d{1,2}[\/|\-]\d{2,4})|(\d{1,2}[\/|\-|\ ](?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[\/\-\ \.\,]*\d{2,4})|((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[\/\-\ \.\,]*\d{1,2}[\/\-\,\ ]*\d{2,4})|((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\/\-\ \.\,]\d{1,2}(?:st|nd|th)[\/\-\ \.\,]*\d{2,4})|(\d{1,2}[\/]+\d{4})|([1|2]\d{3})')
    
    df_dates_first = df_dates[0].str.extractall(r'(?P<month>\d{1,2})[\/|\-](?P<day>\d{1,2})[\/|\-](?P<year>\d{2,4})')
    df_dates_first['year'] = df_dates_first['year'].map(lambda x: 1900+int(x) if int(x) < 1900 else int(x))
    df_dates_first = df_dates_first.astype(int)
    
    df_dates_second = df_dates[1].str.extractall(r'(?P<day>\d{1,2})[\/|\-|\ ](?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z]*[\/\-\ \.\,]*(?P<year>\d{2,4})')
    df_dates_second['month'] = df_dates_second['month'].map(lambda x: 1 if x == 'Jan' else x)
    df_dates_second['month'] = df_dates_second['month'].map(lambda x: 2 if x == 'Feb' else x)
    df_dates_second['month'] = df_dates_second['month'].map(lambda x: 3 if x == 'Mar' else x)
    df_dates_second['month'] = df_dates_second['month'].map(lambda x: 4 if x == 'Apr' else x)
    df_dates_second['month'] = df_dates_second['month'].map(lambda x: 5 if x == 'May' else x)
    df_dates_second['month'] = df_dates_second['month'].map(lambda x: 6 if x == 'Jun' else x)
    df_dates_second['month'] = df_dates_second['month'].map(lambda x: 7 if x == 'Jul' else x)
    df_dates_second['month'] = df_dates_second['month'].map(lambda x: 8 if x == 'Aug' else x)
    df_dates_second['month'] = df_dates_second['month'].map(lambda x: 9 if x == 'Sep' else x)
    df_dates_second['month'] = df_dates_second['month'].map(lambda x: 10 if x == 'Oct' else x)
    df_dates_second['month'] = df_dates_second['month'].map(lambda x: 11 if x == 'Nov' else x)
    df_dates_second['month'] = df_dates_second['month'].map(lambda x: 12 if x == 'Dec' else x)
    df_dates_second = df_dates_second.astype(int)
    
    df_dates_third = df_dates[2][194:228].str.extractall(r'(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[\/\-\ \.\,]*(?P<day>\d{2})[\/\-\ \.\,]*(?P<year>\d{2,4})')
    df_dates_third['month'] = df_dates_third['month'].map(lambda x: 1 if x == 'Jan' else x)
    df_dates_third['month'] = df_dates_third['month'].map(lambda x: 2 if x == 'Feb' else x)
    df_dates_third['month'] = df_dates_third['month'].map(lambda x: 3 if x == 'Mar' else x)
    df_dates_third['month'] = df_dates_third['month'].map(lambda x: 4 if x == 'Apr' else x)
    df_dates_third['month'] = df_dates_third['month'].map(lambda x: 5 if x == 'May' else x)
    df_dates_third['month'] = df_dates_third['month'].map(lambda x: 6 if x == 'Jun' else x)
    df_dates_third['month'] = df_dates_third['month'].map(lambda x: 7 if x == 'Jul' else x)
    df_dates_third['month'] = df_dates_third['month'].map(lambda x: 8 if x == 'Aug' else x)
    df_dates_third['month'] = df_dates_third['month'].map(lambda x: 9 if x == 'Sep' else x)
    df_dates_third['month'] = df_dates_third['month'].map(lambda x: 10 if x == 'Oct' else x)
    df_dates_third['month'] = df_dates_third['month'].map(lambda x: 11 if x == 'Nov' else x)
    df_dates_third['month'] = df_dates_third['month'].map(lambda x: 12 if x == 'Dec' else x)
    df_dates_third = df_dates_third.astype(int)
    
    df_dates_six = df_dates[2][228:343].str.extractall(r'(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[\/\-\ \.\,]*(?P<year>\d{2,4})')
    df_dates_six['day'] = 1
    df_dates_six['month'] = df_dates_six['month'].map(lambda x: 1 if x == 'Jan' else x)
    df_dates_six['month'] = df_dates_six['month'].map(lambda x: 2 if x == 'Feb' else x)
    df_dates_six['month'] = df_dates_six['month'].map(lambda x: 3 if x == 'Mar' else x)
    df_dates_six['month'] = df_dates_six['month'].map(lambda x: 4 if x == 'Apr' else x)
    df_dates_six['month'] = df_dates_six['month'].map(lambda x: 5 if x == 'May' else x)
    df_dates_six['month'] = df_dates_six['month'].map(lambda x: 6 if x == 'Jun' else x)
    df_dates_six['month'] = df_dates_six['month'].map(lambda x: 7 if x == 'Jul' else x)
    df_dates_six['month'] = df_dates_six['month'].map(lambda x: 8 if x == 'Aug' else x)
    df_dates_six['month'] = df_dates_six['month'].map(lambda x: 9 if x == 'Sep' else x)
    df_dates_six['month'] = df_dates_six['month'].map(lambda x: 10 if x == 'Oct' else x)
    df_dates_six['month'] = df_dates_six['month'].map(lambda x: 11 if x == 'Nov' else x)
    df_dates_six['month'] = df_dates_six['month'].map(lambda x: 12 if x == 'Dec' else x)
    df_dates_six = df_dates_six.astype(int)
      
    df_dates_fourth = df_dates[4].str.extractall(r'(?P<month>\d{1,2})[\/]+(?P<year>\d{4})')
    df_dates_fourth['day'] = 1
    df_dates_fourth = df_dates_fourth.astype(int)
        
    df_dates_fith = df_dates[5].str.extractall(r'(?P<year>[1|2]\d{3})')
    df_dates_fith['month'] = 1
    df_dates_fith['day'] = 1  
    df_dates_fith = df_dates_fith.astype(int)
            
    df_dates_final = pd.concat((df_dates_first, df_dates_second), axis=0, join='outer')
    df_dates_final = pd.concat((df_dates_final, df_dates_third), axis=0, join='outer')
    df_dates_final = pd.concat((df_dates_final, df_dates_six), axis=0, join='outer')
    df_dates_final = pd.concat((df_dates_final, df_dates_fourth), axis=0, join='outer')
    df_dates_final = pd.concat((df_dates_final, df_dates_fith), axis=0, join='outer')
    df_dates_final.sort_values(['year', 'month', 'day'], inplace=True)
    df_dates_final = df_dates_final.reset_index('match')
    answer = pd.Series(df_dates_final.index)
            
    return answer

In [3]:
date_sorter()

0        9
1       84
2        2
3       53
4       28
      ... 
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64